In [1]:
# import libraries to scrape websites
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np

In [2]:
# read the website
driver = webdriver.Chrome()
website = "https://unfccc.int/process/conferences/pastconferences/durban-climate-change-conference-november-2011/statements-and-resources/Statements"
driver.get(website)

In [3]:
# scrape list of speeches
speeches = driver.find_elements(By.XPATH, '//td[@valign="top"]/p/strong')
speeches_list = []
for speech in range(len(speeches)):
    speeches_list.append(speeches[speech].text)

In [8]:
del speeches_list[0]

In [5]:
# scrape links 
links = driver.find_elements(By.XPATH, '//td[@valign="top"]/p/a')
links_list = []
for link in links:
    links_list.append(link.get_attribute('href'))

In [11]:
# store data in to a dataframe
data = list(zip(speeches_list, links_list))
columns = ['Speech', 'Link']
df1 = pd.DataFrame(data, columns=columns)

In [12]:
# scrape High Level Segment with statements by Heads of States and Governments
# scrape list of speeches
speeches = driver.find_elements(By.XPATH, '//td/p/a | //td[@width="33%"]/p/a')
speeches_list = []
for speech in range(len(speeches)):
    speeches_list.append(speeches[speech].text)

In [15]:
# scrape links 
links = driver.find_elements(By.XPATH, '//td/p/a | //td[@width="33%"]/p/a')
links_list = []
for link in links:
    links_list.append(link.get_attribute('href'))

In [17]:
# store data in to a dataframe
data = list(zip(speeches_list, links_list))
columns = ['Speech', 'Link']
df2 = pd.DataFrame(data, columns=columns)

In [20]:
# clean df2 by dropping the statements and spanish version
df2 = df2.drop([0,1,2,86], axis=0)

In [21]:
df = pd.concat([df1, df2], axis=0)

In [22]:
df.to_excel('UNFCC COP 17 Links.xlsx')

In [23]:
# set directory to save the pdfs
import os
os.chdir("C:/Users/Wei'En/Documents/MSE Text Analytics/UNFCC COP17 PDF/")

In [24]:
# download pdfs
import urllib3
urllib3.disable_warnings()

for link in df['Link']:
    filename = link.split("/")[-1]
    with urllib3.PoolManager() as http:
        r = http.request('GET', link)
        with open(filename, 'wb') as fout:
            fout.write(r.data)

In [25]:
# import libraries to convert pdfs to text
import io
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import sys, getopt

In [26]:
#converts pdf, returns its text content as a string
def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = io.StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text 

#converts all pdfs in directory pdfDir, saves all resulting txt files to txtdir
def convertMultiple(pdfDir, txtDir):
    if pdfDir == "": pdfDir = os.getcwd() + "\\" #if no pdfDir passed in 
    for pdf in os.listdir(pdfDir): #iterate through pdfs in pdf directory
        fileExtension = pdf.split(".")[-1]
        if fileExtension == "pdf" or fileExtension == "PDF":
            pdfFilename = pdfDir + pdf 
            text = convert(pdfFilename) #get string of text content of pdf
            textFilename = txtDir + pdf + ".txt"
            textFile = open(textFilename, "w", encoding="utf-8") #make text file
            textFile.write(text) #write text to text file

In [27]:
pdfDir = "C:/Users/Wei'En/Documents/MSE Text Analytics/UNFCC COP17 PDF/"
txtDir = "C:/Users/Wei'En/Documents/MSE Text Analytics/UNFCC COP17 Text/"
convertMultiple(pdfDir, txtDir)